In [1]:
import numpy as np
import torch, torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline
import data
import pandas as pd

[nltk_data] Downloading package punkt to /home/iris/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load Data

In [22]:
# Load ChAII dataset
import json
import random
import pandas as pd
from pathlib import Path

pd.set_option("display.max_rows", 20, "display.max_columns", None)

data_path = Path("chaii-hindi-and-tamil-question-answering/")
json_dicts = []

def get_dataframe(file_path):
    df = pd.DataFrame()
    with open(file_path,'r') as f:
        df = pd.read_csv(f)
    df = df.astype(str)
    df = df.apply(lambda x: x.str.strip())
    return df

train_data = get_dataframe(data_path / "train.csv")
test_data = get_dataframe(data_path / "test.csv")
test_data

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [39]:
train_data[:1]

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil


In [38]:
train_data[:1]['context'][0][53]

'2'

### Convert Data

In [25]:
def convert_to_qa_format_kaggle(row):
    answer = {}
    try:
        answer["text"] = row["answer_text"]
        answer["answer_start"] = int(row["answer_start"])
    except:
        answer["text"] = ''
        answer["answer_start"] = -1
    qa_json = {
        "title": "",
        "paragraphs": [
            {
                "context": row["context"],
                "qas": [
                    {
                        "question": row["question"],
                        "id": row["language"] + '-' + str(row["id"]),
                        "answers": [answer]
                    }
                ]
            }
        ],
    }
    
    return qa_json

# Process one language at a time
# Here chaii_data is a pandas dataframe
def get_qa_data_from_kaggle_format(chaii_data, language):
    qa_data = {"data":[], "version":f"chaii_{language}"}
    for index, row in chaii_data.iterrows():
        if row["language"] == language:
            qa_datapoint = convert_to_qa_format_kaggle(row)
            qa_data["data"].append(qa_datapoint)

    print("QA (SQuAD) format:")
    pprint(qa_data["data"][0])
    return qa_data

hi_test_qa_data = get_qa_data_from_kaggle_format(test_data, 'hindi')
ta_test_qa_data = get_qa_data_from_kaggle_format(test_data, 'tamil')

hi_train_qa_data = get_qa_data_from_kaggle_format(train_data, 'hindi')
ta_train_qa_data = get_qa_data_from_kaggle_format(train_data, 'tamil')

QA (SQuAD) format:
{'paragraphs': [{'context': 'ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, '
                            'महाराष्ट्र) एक भारतीय बैडमिंटन खिलाडी हैं। \n'
                            ' प्रारंभिक जीवन \n'
                            'ज्वाला गुट्टा का जन्म 7 सितंबर 1983 को वर्धा, '
                            'महाराष्ट्र में हुआ था। उनके पिता एम. क्रांति '
                            'तेलुगु और मां येलन चीन से हैं। उनकी मां येलन '
                            'गुट्टा पहली बार 1977 में अपने दादा जी के साथ भारत '
                            'आई थीं। ज्वाला गुट्टा की प्रारंभिक पढ़ाई हैदराबाद '
                            'से हुई और यहीं से उन्होंने बैडमिंटन खेलना भी शुरू '
                            'किया। \n'
                            ' कॅरियर \n'
                            '10 साल की उम्र से ही ज्वाला गुट्टा ने एस.एम. आरिफ '
                            'से ट्रेनिंग लेना शुरू कर दिया था। एस.एम. आरिफ '
                            'भारत के जाने माने खेल प्रशिक्षक हैं जिन्हें

In [27]:
# Splitting data into train and dev and saving converted QA formats
def split_data(qa_data, test_qa_data, lang_code):
    split_data_path = Path("/home/iris/paulshab/KaggleComp/qa_comp/chaii-hindi-and-tamil-question-answering/chaii_data/")
    !mkdir /kaggle/working/chaii_data

    test_qa_data_datapoints = test_qa_data["data"]
    test_qa_data = {"data": test_qa_data_datapoints, "version":f"chaii_{lang_code}_test"}
    
    qa_data_datapoints = qa_data["data"]
    random.shuffle(qa_data_datapoints)
    
    train_size = int(len(qa_data_datapoints)*0.8)
    train_qa_data_datapoints, dev_qa_data_datapoints = qa_data_datapoints[:train_size], qa_data_datapoints[train_size:]
    
    train_qa_data = {"data":train_qa_data_datapoints, "version":f"chaii_{lang_code}_train"}
    dev_qa_data = {"data":dev_qa_data_datapoints, "version":f"chaii_{lang_code}_dev"}

    with open(split_data_path / f"test.{lang_code}.qa.jsonl",'w') as f:
        json.dump(test_qa_data,f)
    
    with open(split_data_path / f"dev.{lang_code}.qa.jsonl",'w') as f:
        json.dump(dev_qa_data,f)

    with open(split_data_path / f"test.{lang_code}.qa.jsonl",'w') as f:
        json.dump(train_qa_data, f)

    print(f"{lang_code} Test data size: %d" % len(test_qa_data_datapoints))
    print(f"{lang_code} Dev data size: %d" % len(dev_qa_data_datapoints))
    print(f"{lang_code} Train data size: %d" % len(train_qa_data_datapoints))
    
split_data(hi_train_qa_data, hi_test_qa_data, 'hi')
split_data(ta_train_qa_data, ta_test_qa_data, 'ta')

mkdir: cannot create directory ‘/kaggle/working/chaii_data’: No such file or directory
hi Test data size: 3
hi Dev data size: 150
hi Train data size: 596
mkdir: cannot create directory ‘/kaggle/working/chaii_data’: No such file or directory
ta Test data size: 2
ta Dev data size: 74
ta Train data size: 294


In [19]:
import json
my_path = "/home/iris/paulshab/KaggleComp/qa_comp/chaii-hindi-and-tamil-question-answering/chaii_data/dev.ta.qa.jsonl"

with open(my_path) as json_file:
    data = json.load(json_file)



{'data': [{'title': '',
   'paragraphs': [{'context': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின்வரும் 206 (மார்பெலும்பு மூன்று பகுதிகளாகக் கருதப்பட்டால் 208) எண்ணிக்கையான எலும்புகளைக் கொண்டிருக்கும். இந்த எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் பொறுத்து மாறுபடக்கூடும். எடுத்துக்காட்டாக, மிகக் குறைந்த எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு மேலதிகமான கீழ் முதுகெலும்பு காணப்படுவதுண்டு; இணைந்த சில எலும்புகளைத் தனி எலும்பாகக் கருதாவிடின், ஐந்து இணைந்த திருவெலும்பு; மூன்று (3 - 5) குயிலலகு எலும்புகள் சேர்ந்து 26 எண்ணிக்கையிலான முதுகெலும்புகள் 33 ஆகக் கருதப்படலாம்.\nமனித மண்டையோட்டில் 22 எலும்புகள் (காதுச் சிற்றென்புகளைத் தவிர) உள்ளன; இவை எட்டு மண்டையறை (cranium) எலும்புகளாகவும் 14 முக எலும்புகளாகவும் (facial bones) பிரிக்கப்பட்டுள்ளன. (தடித்த எண்கள் அருகிலுள்ள படத்தில் காணும் எண்களைக் குறிக்கின்றன.)\n\n\n\nமண்டையறை எலும்புகள் (8)\n 1 நுதலெலும்பு (frontal bone) \n 2 சுவரெலும்பு (parietal bone) (2)\n 3 கடைநுதலெலும்பு (temporal bone) (2)\n 4 பிடர் எலும்பு

In [20]:
pprint(data)

{'data': [{'paragraphs': [{'context': 'ஒரு சாதாரண வளர்ந்த மனிதனுடைய '
                                      'எலும்புக்கூடு பின்வரும் 206 '
                                      '(மார்பெலும்பு மூன்று பகுதிகளாகக் '
                                      'கருதப்பட்டால் 208) எண்ணிக்கையான '
                                      'எலும்புகளைக் கொண்டிருக்கும். இந்த '
                                      'எண்ணிக்கை உடற்கூட்டியல் வேறுபாடுகளைப் '
                                      'பொறுத்து மாறுபடக்கூடும். '
                                      'எடுத்துக்காட்டாக, மிகக் குறைந்த '
                                      'எண்ணிக்கையான மனிதர்களில், ஒரு மேலதிக '
                                      'விலா எலும்பு (கழுத்துவில்) அல்லது ஒரு '
                                      'மேலதிகமான கீழ் முதுகெலும்பு '
                                      'காணப்படுவதுண்டு; இணைந்த சில '
                                      'எலும்புகளைத் தனி எலும்பாகக் '
                                      'கருதாவிடின், ஐந்து